In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('COMPAS_preprocessed.csv')
#cols =  ['race','age', 'c_charge_degree', 'race', 'score_text', 'sex',
 #      'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid',
  #     'two_year_recid', 'days_in_jail']
#df = df[cols]
from sklearn.model_selection import train_test_split
col_X = ['race','age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']
X = df[col_X]
y = df['two_year_recid']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

X_pos = X_train[X_train['race']==1]
X_neg = X_train[X_train['race']==0]

y_pos = np.array(y_train[X_train['race']==1])
y_neg = np.array(y_train[X_train['race']==0])

col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']

X_pos = np.array(X_pos[col_X])
X_neg = np.array(X_neg[col_X])

In [ ]:
X_pos

,age,c_charge_degree,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,days_in_jail
990,38,0,1,0,14,-1.0,10,1,4
1614,36,1,-1,1,3,-1.0,2,1,2
3784,19,0,-1,1,1,-1.0,4,1,3
1739,37,0,0,1,9,0.0,6,1,1
5027,23,1,0,0,2,-1.0,7,1,1
...,...,...,...,...,...,...,...,...,...
466,35,1,1,1,1,-1.0,9,1,79
3092,23,0,1,1,3,-1.0,8,1,0
5191,27,0,0,0,3,0.0,6,1,2
5226,44,0,0,1,11,-1.0,5,1,1


In [ ]:
# Equation 12

def distances(X, v, alpha, N, D, K):
    dists = np.zeros((N, D))
    for n in range(N):
        for i in range(D):
            for k in range(K):
                dists[n, k] += (X[n, i] - v[k, i]) * (X[n, i] - v[k, i]) * alpha[i]
    return dists



In [ ]:
# Equation 2&3
def M_nk(dists, N, K):
    M_nk = np.zeros((N, K))
    exp = np.zeros((N, K))
    denom = np.zeros(N)
    for n in range(N):
        for j in range(K):
            exp[n, j] = np.exp(-1 * dists[n, j])
            denom[n] += exp[n, j]
        for j in range(K):
            if denom[n]:
                M_nk[n, j] = exp[n, j] / denom[n]
            else:
                M_nk[n, j] = exp[n, j] / 1e-6
    return M_nk

In [ ]:
# Equation 6
def M_k(M_nk, N, K):
    M_k = np.zeros(K)
    for j in range(K):
        for n in range(N):
            M_k[j] += M_nk[n, j]
        M_k[j] /= N
    return M_k

In [ ]:
# Equation 8&9
def Loss_x(X, M_nk, v, N, D, K):
    x_n_hat = np.zeros((N, D))
    L_x = 0.0
    for n in range(N):
        for i in range(D):
            for k in range(K):
                x_n_hat[n, i] += M_nk[n, k] * v[k, i]
            L_x += (X[n, i] - x_n_hat[n, i]) * (X[n, i] - x_n_hat[n, i])
    return x_n_hat, L_x

In [ ]:
# Equation 10&11
def Loss_y(M_nk, y, w, N, K):
    yhat = np.zeros(N)
    L_y = 0.0
    for n in range(N):
        for k in range(K):
            yhat[n] += M_nk[n, k] * w[k]
        yhat[n] = 1e-6 if yhat[n] <= 0 else yhat[n]
        yhat[n] = 0.999 if yhat[n] >= 1 else yhat[n]
        L_y += -1 * y[n] * np.log(yhat[n]) - (1.0 - y[n]) * np.log(1.0 - yhat[n])
    return yhat, L_y

In [ ]:

# Equation 4 (target function)
def LFR(params, X_pos, X_neg, y_pos, y_neg, K, A_z, A_x, A_y, results=0):   
    #LFR.iters += 1
    N_pos, D = X_pos.shape # protected set
    N_neg, _ = X_neg.shape # non-protected set
    
    #print('PARAMS SHAPE: ', params.shape)
    alpha_pos = params[:D]
    alpha_neg = params[D: 2 * D]
    w = params[2 * D: (2 * D) + K]
    #print(alpha_pos.shape, alpha_neg.shape, w.shape)
    v = np.matrix(params[(2 * D) + K:]).reshape((K, D)) # set of prototypes

    dists_pos = distances(X_pos, v, alpha_pos, N_pos, D, K)
    dists_neg = distances(X_neg, v, alpha_neg, N_neg, D, K)

    M_nk_pos = M_nk(dists_pos, N_pos, K)
    M_nk_neg = M_nk(dists_neg, N_neg, K)

    M_k_pos = M_k(M_nk_pos, N_pos, K)
    M_k_neg = M_k(M_nk_neg, N_neg, K)

    L_z = 0.0
    for k in range(K):
        L_z += abs(M_k_pos[k] - M_k_neg[k]) # Equation 7

    x_n_hat_pos, L_x_pos = Loss_x(X_pos, M_nk_pos, v, N_pos, D, K)
    x_n_hat_neg, L_x_neg = Loss_x(X_neg, M_nk_neg, v, N_neg, D, K)
    L_x = L_x_pos + L_x_neg

    yhat_pos, L_y_pos = Loss_y(M_nk_pos, y_pos, w, N_pos, K)
    yhat_neg, L_y_neg = Loss_y(M_nk_neg, y_neg, w, N_neg, K)
    L_y = L_y_pos + L_y_neg

    criterion = A_x * L_x + A_y * L_y + A_z * L_z

    if results != 0:
        return yhat_pos, yhat_neg, M_nk_pos, M_nk_neg
    else:
        return criterion

In [ ]:
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA
D = X_pos.shape[1]
K = 5
alpha_pos = np.random.uniform(size=D)
alpha_neg = np.random.uniform(size=D)
w = np.random.uniform(size=K) 
v = np.random.uniform(size=K*D)
params = alpha_pos
for item in [alpha_neg, w, v]:
  params = np.append(params, item.flatten())
scipy.optimize.minimize(LFR, 
                        x0=params, 
                        args=(X_pos, X_neg, y_pos, y_neg, K, 1000, 1e-4, 0.1),
                        method='SLSQP')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


     fun: 2292.3930332228624
     jac: array([-4.89501953e-02,  3.66210938e-04, -3.05175781e-05, -9.15527344e-05,
       -6.10351562e-04,  2.10266113e-02, -6.68334961e-03, -1.83105469e-04,
        9.49096680e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.22070312e-04,
        0.00000000e+00,  1.26752625e+01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.31530762e-02, -1.22070312e-04,  5.18798828e-04,  2.13623047e-04,
        9.15527344e-04, -2.24304199e-02, -9.15527344e-05,  9.76562500e-04,
        1.02233887e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.14413147e+01,  2.